In [1]:
# https://www.youtube.com/watch?v=O7GXjLNo_m8&ab_channel=akkimotivational
import pandas as pd
import numpy as np
import warnings
import pymysql
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
df = pd.read_csv('./often/car.csv', encoding='unicode_escape')
df.head(2)

,Name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Alto 800 LXI Opt,2023,410000,10000,Petrol,Individual,Manual,First Owner,19.03 kmpl,999 CC,71.01bhp,96Nm,5.0
1,Skoda Slavia 1.0 TSI Ambition,2023,1350000,10000,Petrol,Individual,Manual,First Owner,14.08 kmpl,1956 CC,167.67bhp,350nm,5.0


In [2]:
df.mileage = df.mileage.str.split(' ').str[0]
df.engine = df.engine.str.split(' ').str[0]
df.max_power = df.max_power.str.split('b').str[0]
df.head(2)

,Name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Alto 800 LXI Opt,2023,410000,10000,Petrol,Individual,Manual,First Owner,19.03,999,71.01,96Nm,5.0
1,Skoda Slavia 1.0 TSI Ambition,2023,1350000,10000,Petrol,Individual,Manual,First Owner,14.08,1956,167.67,350nm,5.0


In [3]:
for i in range(df.shape[1]):
    df[df.columns[i]] = df[df.columns[i]].astype(str)
    df[df.columns[i]] = df[df.columns[i]].str.replace('(','') 
    df[df.columns[i]] = df[df.columns[i]].str.replace(')','') 

In [4]:
str_cols = ['Name', 'fuel','seller_type','transmission','owner','torque']
int_cols = ['year', 'selling_price','km_driven','engine','seats']
float_cols = ['mileage', 'max_power']

for i in range(len(str_cols)):
    df[str_cols[i]] = df[str_cols[i]].astype(str)
    
for i in range(len(int_cols)):
    
    df[int_cols[i]] = df[int_cols[i]].astype(float)  
    df[int_cols[i]] = df[int_cols[i]].fillna(0)
    df[int_cols[i]] = df[int_cols[i]].astype(int)    

for i in range(len(float_cols)):
    df[float_cols[i]] = df[float_cols[i]].fillna('0')
    df[float_cols[i]] = df[float_cols[i]].str.replace(' ','0')
    df[float_cols[i]] = df[float_cols[i]].astype(float)

print(df.dtypes)

Name              object
year               int32
selling_price      int32
km_driven          int32
fuel              object
seller_type       object
transmission      object
owner             object
mileage          float64
engine             int32
max_power        float64
torque            object
seats              int32
dtype: object


In [5]:
distinct = df[['Name', 'fuel']]
distinct = distinct.drop_duplicates()
df = df.drop(columns='fuel')
passwd = pd.read_csv('./often/pd.txt')

In [6]:
connect_db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd=passwd.columns[0], charset='utf8', db='car')

with connect_db.cursor() as cursor:
    sql = """
    CREATE TABLE IF NOT EXISTS raw(
        ID int NOT NULL AUTO_INCREMENT PRIMARY KEY,
        
        Name varchar(99),
        seller_type varchar(99),
        transmission varchar(99),
        owner varchar(99),
        torque varchar(99),
        
        year int,
        selling_price int,
        km_driven int,
        engine int,
        seats int,
        
        mileage float,
        max_power float
    );
    """
    cursor.execute(sql)
    connect_db.commit()
    connect_db.close()
    
connect_db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd=passwd.columns[0], charset='utf8mb4', db='car')
with connect_db.cursor() as cursor:
    for i in range(len(df)): #len(df)
        sql = """
        INSERT INTO raw (Name, year, selling_price, km_driven, seller_type,
       transmission, owner, mileage, engine, max_power, torque,
       seats) VALUES (
        '{}', '{}', '{}', '{}','{}', '{}','{}', '{}','{}', '{}',
        '{}', '{}'

)""".format(
        df.iloc[i][0], df.iloc[i][1], df.iloc[i][2], df.iloc[i][3], df.iloc[i][4], df.iloc[i][5], df.iloc[i][6], df.iloc[i][7], df.iloc[i][8], df.iloc[i][9],
        df.iloc[i][10], df.iloc[i][11]
)

        cursor.execute(sql)
        connect_db.commit()
    print(sql)

    connect_db.close()


        INSERT INTO raw (Name, year, selling_price, km_driven, seller_type,
       transmission, owner, mileage, engine, max_power, torque,
       seats) VALUES (
        'Mahindra Willys CJ 3B 4X4', '1983', '300000', '10000','Individual', 'Manual','Third Owner', 'nan','0', 'nan',
        'nan', '0'

)


In [7]:
connect_db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd=passwd.columns[0], charset='utf8', db='car')

with connect_db.cursor() as cursor:
    sql = """
    CREATE TABLE IF NOT EXISTS dist(
        ID int NOT NULL AUTO_INCREMENT PRIMARY KEY,
        
        Name varchar(99),
        fuel varchar(99)
    );
    """
    cursor.execute(sql)
    connect_db.commit()
    connect_db.close()
    
connect_db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd=passwd.columns[0], charset='utf8mb4', db='car')
with connect_db.cursor() as cursor:
    for i in range(len(distinct)): #len(distinct)
        sql = """
        INSERT INTO dist (Name, fuel) VALUES (
        '{}', '{}'

)""".format(
        distinct.iloc[i][0], distinct.iloc[i][1]
)

        cursor.execute(sql)
        connect_db.commit()
    print(sql)

    connect_db.close()


        INSERT INTO dist (Name, fuel) VALUES (
        'Mahindra Willys CJ 3B 4X4', 'Diesel'

)
